In [1]:
!pip uninstall -y allennlp
!pip install "torch==1.13.1+cu117" --extra-index-url https://download.pytorch.org/whl/cu117 --upgrade --quiet
!pip install -q "transformers==4.27.4" "datasets==1.16.0" huggingface_hub peft
!pip install -q "s3fs==2021.10" "fsspec==2021.10"
!pip install -U accelerate>=0.21.0

# !git clone https://github.com/NVIDIA/apex
# !cd apex && pip install -q -v --disable-pip-version-check --no-cache-dir --no-build-isolation --config-settings "--build-option=--cpp_ext" --config-settings "--build-option=--cuda_ext" ./
# !cd apex && pip install -q -v --disable-pip-version-check --no-cache-dir --no-build-isolation --global-option="--cpp_ext" --global-option="--cuda_ext" ./

Found existing installation: allennlp 1.2.0
Uninstalling allennlp-1.2.0:
  Successfully uninstalled allennlp-1.2.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

fastai 2.0.18 requires torch<1.7,>=1.6, but you'll have torch 1.13.1+cu117 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

fastai 2.0.18 requires torch<1.7,>=1.6, but you'll have torch 1.13.1+cu117 which is incompatib

In [2]:
from datasets import load_dataset
import torch, transformers
torch.__version__, transformers.__version__

('1.13.1+cu117', '4.27.4')

# New model

In [3]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration 
base_model = "ai-forever/FRED-T5-1.7B"
device='cuda'
tokenizer = GPT2Tokenizer.from_pretrained(base_model, eos_token='</s>')


model = T5ForConditionalGeneration.from_pretrained(base_model)
model.to(device)

# optional
# tokenizer.add_tokens(['<calculator>', '</calculator>', '####'])
# model.resize_token_embeddings(len(tokenizer))

device

'cuda'

# Load LoRA

In [4]:
# from peft import PeftModel

# adapters_name = '/kaggle/input/t5-1-7b-lora/peft-checkpoint-v2'
# model = PeftModel.from_pretrained(model, adapters_name)
# model = model.merge_and_unload()

In [5]:
# q = 'На халат уходит 2 рулона синего волокна и половина этого количества белого волокна. Сколько всего болтов нужно?'
# lm_text=f'<SC6>Задача: {q}\nРешение:<extra_id_0>####<extra_id_1>'
# # lm_text=f'<LM>Задача: {q}\nРешение:'

# input_ids=torch.tensor([tokenizer.encode(lm_text)]).to(device)
# outputs=model.generate(input_ids,
#                             eos_token_id=tokenizer.eos_token_id,
#                             early_stopping=True,
#                             top_p=0.9,
#                             temperature=0.05,
#                             repetition_penalty=2.0,
#                             min_length=10,
#                             max_length=512
#                       )
# output = tokenizer.decode(outputs[0][1:])
# print(output)

# Data processing

In [6]:
import re
import numpy as np

data_files = {"train": "gsm_train.csv", "test": "gsm_test.csv"}
dataset = load_dataset('/kaggle/input/gsm-8k-ru', data_files=data_files).shuffle(seed=0)
train_data = dataset["test"].train_test_split(test_size=0.5) # 50% of test to validation
train_data["train"] = dataset["train"]
train_data


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/gsm-8k-ru-8b519d935238902b/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 5824
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 200
    })
})

In [7]:
def get_target_digits(text):
    text = text.split('####')[-1]
    text = re.sub("[^0-9.,]", "", text)
    text = re.sub("^[.,]|[.,]$", "", text)
    try:
        target = float(text)
    except:
        target = np.NaN
    finally:
        return target

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    final_preds = [get_target_digits(pred) for pred in decoded_preds]
    final_labels = [[get_target_digits(label)] for label in decoded_labels]

    # length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    # answer accuracy
    eps = 1e-6
    prediction_corrects = np.abs(final_preds - final_labels) <= eps
    result["accuracy"] = np.mean(prediction_corrects)
    
    result = {k: round(v, 4) for k, v in result.items()}
    print(result)
    return result

In [8]:
def split_text_by_space(text, index):
    space_before = text.rfind(' ', 0, index)
    space_after = text.find(' ', index)
    if space_before == -1 and space_after == -1:
        return text, ''
    
    if space_before == -1:
        res = text[:space_after], text[space_after + 1:]
    elif space_after == -1:
        res = text[:space_before], text[space_before + 1:]
    else:
        if index - space_before < space_after - index:
            res = text[:space_before], text[space_before + 1:]
        else:
            res = text[:space_after], text[space_after + 1:]
    if np.random.rand() < 0.5:
        return res[0] + ' ', res[1]
    else:
        return res[0], ' ' + res[1]

def build_text(question, answer):
    return f'Задача: {question}\nРешение:{answer}'

denoiser_params = {
    '<SC1>': (3, 0.15),
    '<SC2>': (8, 0.15),
    '<SC3>': (64, 0.15),
    '<SC4>': (3, 0.5),
    '<SC5>': (8, 0.5)
}

def generate_denoise_input(questions, answers):
    denoisers = np.random.choice(['<LM>', '<SC1>', '<SC2>', '<SC3>', '<SC4>', '<SC5>', '<SC6>'], 
                                 size=len(questions), p=[0.3, 0.05, 0.05, 0.05, 0.05, 0.05, 0.45])
    inputs, outputs = [], []
    for question, answer, denoiser in zip(questions, answers, denoisers):
        answer = answer.replace('<calculator>', '[[').replace('</calculator>', ']]')
        if denoiser == '<SC6>':
            answer = answer.split('####')
            inputs.append(f'<SC6>Задача: {question}\nРешение:<extra_id_0>####<extra_id_1>')
            outputs.append(f'<extra_id_0>{answer[0]}<extra_id_1>{answer[1]}</s>')
        elif denoiser == '<LM>':
            text = build_text(question, answer)
            loc = len(text) * 0.75
            scale = len(text) * 0.03
            split_index = int(np.random.normal(loc=loc, scale=scale))
            text = split_text_by_space(text, split_index)
            inputs.append(f'<LM>{text[0]}')
            outputs.append(f'{text[1]}</s>')
        else:
            text = build_text(question, answer)
            mu, r = denoiser_params[denoiser]
            loc = mu * 3
            scale = loc * 0.1
            replaced_text = ''
            mask_id = 0
            masked_text = ''
            skipped = True
            while len(text) != 0:
                span_size = int(np.random.normal(loc=loc, scale=scale))
                if len(text) <= span_size:
                    span, text = text, ''
                else:
                    span, text = split_text_by_space(text, span_size)
                if skipped:
                    skipped = False
                    replaced_text += span
                else:
                    skipped = True
                    if np.random.rand() <= 2*r:
                        replaced_text += f'<extra_id_{mask_id}>'
                        masked_text += f'<extra_id_{mask_id}>{span}'
                        mask_id += 1
                    else:
                        replaced_text += span
            inputs.append(f'{denoiser}{replaced_text}')
            outputs.append(f'{masked_text}</s>')
    return inputs, outputs


In [9]:
sustem_prompt = '''Ты эксперт в решении математических задач. Твоя задача: решить задачу 
расписывая решение по шагам и выполняя одно арифметическое действие за раз. Тебе необходимо 
пользоваться калькулятором вместо того, чтобы считать результат арифметического выражения. 
Для этого помести математическое выражение между тэгами <calculator> и </calculator>, чтобы вызвать калькулятор.
Результат выражения будет вычислен при помощи калькулятора и записан сразу же после вызова (после закрывающего тэга). 
Используй "+" для сложения, "-" для вычитания, "*" для умножения, "/" для деления чисел. 
Финальный ответ напиши в самом конце после "####". Подумай хорошо и отвечай как можно лучше.
'''.replace('\n', ' ').strip()
# NOTE: большой контекст не позволяет учиться с большим батчем
sustem_prompt = '' 


def preprocess_function(examples):
    inputs, outputs = generate_denoise_input(examples["question"], examples["answer"])
    
    model_inputs = tokenizer(inputs, text_target=outputs, max_length=338, truncation=True)
    return model_inputs

In [10]:
N = 5
train_data['train'] = train_data['train'].map(lambda examples: {k: v * N for k, v in examples.items()}, batched=True).shuffle()
train_data

DatasetDict({
    train: Dataset({
        features: ['answer', 'question'],
        num_rows: 29120
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 200
    })
})

In [11]:
tokenized_train_data = train_data.map(preprocess_function, batched=True)

In [12]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [13]:
import os
os.environ["WANDB_DISABLED"] = "true"

# Full SFT

In [14]:
# training_args = Seq2SeqTrainingArguments(
#     output_dir="trained",
#     overwrite_output_dir=True,
#     num_train_epochs=1,
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=1,
#     gradient_accumulation_steps = 1,
#     learning_rate=2e-5,
#     save_steps=5_000,
#     save_total_limit=2,
#     do_train = True,
#     do_eval = True,
#     fp16=True
# )

In [15]:
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train_data["train"],
#     eval_dataset=tokenized_train_data["test"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics
# )

# trainer.train()

# LoRA

In [16]:
# from peft import LoraConfig, get_peft_model, TaskType
# from peft.utils.other import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING 
# from transformers import TrainingArguments, Trainer

# lora_config = LoraConfig(
#     r=32,
#     lora_alpha=32,
#     target_modules=TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING["t5"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type=TaskType.SEQ_2_SEQ_LM
# )

# model.enable_input_require_grads()
# peft_model = get_peft_model(model, lora_config)
# # peft_model.to(device)

# def print_number_of_trainable_model_parameters(model):
#     trainable_model_params = 0
#     all_model_params = 0
#     for _, param in model.named_parameters():
#         all_model_params += param.numel()
#         if param.requires_grad:
#             trainable_model_params += param.numel()
#     return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

# print(print_number_of_trainable_model_parameters(peft_model))

In [17]:
# pretrained
from peft import PeftModel
from peft.tuners.lora import mark_only_lora_as_trainable

adapters_name = '/kaggle/input/t5-1-7b-lora/peft-checkpoint-v2'
model.enable_input_require_grads()
peft_model = PeftModel.from_pretrained(model, adapters_name, is_trainable=True)
# peft_model = peft_model.merge_and_unload()
# peft_model._mark_only_adapters_as_trainable()
mark_only_lora_as_trainable(peft_model)

# Train 1

In [18]:
# import os
# # os.environ["WANDB_DISABLED"] = "false"

# # tips https://huggingface.co/docs/transformers/perf_train_gpu_one
# # Parameters https://huggingface.co/docs/transformers/v4.25.1/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments
# train_batch_size = 16
# eval_batch_size = 16

# training_args = Seq2SeqTrainingArguments(
#     num_train_epochs = 5, # 25
#     learning_rate = 3e-4,
#     lr_scheduler_type = "linear",
#     weight_decay = 1e-3,
#     warmup_steps = 50,
#     optim="adafactor",
# #     optim="adamw_apex_fused",
    
# #     auto_find_batch_size=True,
#     per_device_train_batch_size=train_batch_size,
#     per_device_eval_batch_size=eval_batch_size, 
#     gradient_checkpointing = True,
#     gradient_accumulation_steps = max(1, train_batch_size // 32),
#     eval_accumulation_steps = max(1, 64 // eval_batch_size),
    
#     output_dir="/kaggle/working/lora_trained",
#     logging_dir='/kaggle/working/training_logs',  
#     overwrite_output_dir=True,
#     save_steps=1800, # at the end of training
#     save_total_limit = 1,
#     do_train = True,
#     do_eval = True,
#     evaluation_strategy = 'steps',
#     logging_steps=100,# 100
    
#     fp16 = False, # it may cause training errors: https://discuss.huggingface.co/t/training-loss-0-0-validation-loss-nan/27950/3
# #     jit_mode_eval = True,
#     seed = 42,
# #     report_to="wandb"
# )



In [19]:
# # for safety background
# for i in range(10):
#     print('checkpoint', i)
#     # reshuffle masked LM
#     print('data processing')
#     tokenized_train_data = train_data.map(preprocess_function, batched=True)
#     trainer = Seq2SeqTrainer(
#         model=peft_model,
#         args=training_args,
#         train_dataset=tokenized_train_data["train"],
#         eval_dataset=tokenized_train_data["test"],
#         tokenizer=tokenizer,
#         data_collator=data_collator,
# #         compute_metrics=compute_metrics
#     )
#     print('training')
#     try:
#         if i == 0:
#             trainer.train()
#         else:
#             trainer.train(resume_from_checkpoint=True)
#     except Exception as ex:
#         print('Training error:', ex)
#     finally:
#         for j, step in enumerate(trainer.state.log_history):
#             print(j, step)
#         peft_model_path=f"/kaggle/working/peft-checkpoint-{i}"

#         trainer.model.save_pretrained(peft_model_path)
#         tokenizer.save_pretrained(peft_model_path)

# Train2

In [20]:
train_batch_size = 16
eval_batch_size = 16

training_args = Seq2SeqTrainingArguments(
    num_train_epochs = 1,
    learning_rate = 3e-4,
    lr_scheduler_type = "linear",
    weight_decay = 0,
    warmup_steps = 50,
    optim="adafactor",
#     optim="adamw_apex_fused",
    
#     auto_find_batch_size=True,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size, 
    gradient_checkpointing = True,
    gradient_accumulation_steps = max(1, train_batch_size // 32),
    eval_accumulation_steps = max(1, 64 // eval_batch_size),
    
    output_dir="/kaggle/working/lora_trained",
    logging_dir='/kaggle/working/training_logs',  
    overwrite_output_dir=True,
    save_steps=1800,
    save_total_limit = 1,
    do_train = True,
    do_eval = True,
    evaluation_strategy = 'steps',
    logging_steps=450,# 100
    
    fp16 = False, # it may cause training errors: https://discuss.huggingface.co/t/training-loss-0-0-validation-loss-nan/27950/3
#     jit_mode_eval = True,
    seed = 42,
#     report_to="wandb"
)




for i in range(3):
    print('checkpoint', i)
    try:
        trainer = Seq2SeqTrainer(
            model=peft_model,
            args=training_args,
            train_dataset=tokenized_train_data["train"],
            eval_dataset=tokenized_train_data["test"],
            tokenizer=tokenizer,
            data_collator=data_collator
        )
        trainer.train()
    except Exception as ex:
        print(ex)
    finally:
        print(trainer.state.log_history)
        peft_model_path=f"peft-checkpoint-{i}"

        trainer.model.save_pretrained(peft_model_path)
        tokenizer.save_pretrained(peft_model_path)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


checkpoint 0


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
450,0.756300,0.663845
900,0.722700,0.656347
1350,0.710800,0.649156
1800,0.708700,0.647571


[{'loss': 0.7563, 'learning_rate': 0.00023220338983050845, 'epoch': 0.25, 'step': 450}, {'eval_loss': 0.663845419883728, 'eval_runtime': 20.8803, 'eval_samples_per_second': 9.578, 'eval_steps_per_second': 0.623, 'epoch': 0.25, 'step': 450}, {'loss': 0.7227, 'learning_rate': 0.0001559322033898305, 'epoch': 0.49, 'step': 900}, {'eval_loss': 0.6563472151756287, 'eval_runtime': 20.8714, 'eval_samples_per_second': 9.582, 'eval_steps_per_second': 0.623, 'epoch': 0.49, 'step': 900}, {'loss': 0.7108, 'learning_rate': 7.966101694915254e-05, 'epoch': 0.74, 'step': 1350}, {'eval_loss': 0.6491555571556091, 'eval_runtime': 20.8823, 'eval_samples_per_second': 9.577, 'eval_steps_per_second': 0.623, 'epoch': 0.74, 'step': 1350}, {'loss': 0.7087, 'learning_rate': 3.389830508474576e-06, 'epoch': 0.99, 'step': 1800}, {'eval_loss': 0.6475707292556763, 'eval_runtime': 20.8846, 'eval_samples_per_second': 9.576, 'eval_steps_per_second': 0.622, 'epoch': 0.99, 'step': 1800}, {'train_runtime': 9768.9274, 'train

Step,Training Loss,Validation Loss
450,0.690700,0.652295
900,0.687800,0.647783
1350,0.685300,0.640844
1800,0.691600,0.638787


[{'loss': 0.6907, 'learning_rate': 0.00023220338983050845, 'epoch': 0.25, 'step': 450}, {'eval_loss': 0.6522954702377319, 'eval_runtime': 20.8737, 'eval_samples_per_second': 9.581, 'eval_steps_per_second': 0.623, 'epoch': 0.25, 'step': 450}, {'loss': 0.6878, 'learning_rate': 0.0001559322033898305, 'epoch': 0.49, 'step': 900}, {'eval_loss': 0.6477830410003662, 'eval_runtime': 20.8805, 'eval_samples_per_second': 9.578, 'eval_steps_per_second': 0.623, 'epoch': 0.49, 'step': 900}, {'loss': 0.6853, 'learning_rate': 7.966101694915254e-05, 'epoch': 0.74, 'step': 1350}, {'eval_loss': 0.6408435702323914, 'eval_runtime': 20.8711, 'eval_samples_per_second': 9.583, 'eval_steps_per_second': 0.623, 'epoch': 0.74, 'step': 1350}, {'loss': 0.6916, 'learning_rate': 3.389830508474576e-06, 'epoch': 0.99, 'step': 1800}, {'eval_loss': 0.6387869715690613, 'eval_runtime': 20.8782, 'eval_samples_per_second': 9.579, 'eval_steps_per_second': 0.623, 'epoch': 0.99, 'step': 1800}, {'train_runtime': 9762.7694, 'trai

Step,Training Loss,Validation Loss
450,0.656700,0.649581
900,0.660400,0.643848
1350,0.665100,0.636238
1800,0.678800,0.633888


[{'loss': 0.6567, 'learning_rate': 0.00023220338983050845, 'epoch': 0.25, 'step': 450}, {'eval_loss': 0.6495810151100159, 'eval_runtime': 20.8817, 'eval_samples_per_second': 9.578, 'eval_steps_per_second': 0.623, 'epoch': 0.25, 'step': 450}, {'loss': 0.6604, 'learning_rate': 0.0001559322033898305, 'epoch': 0.49, 'step': 900}, {'eval_loss': 0.6438479423522949, 'eval_runtime': 20.8671, 'eval_samples_per_second': 9.584, 'eval_steps_per_second': 0.623, 'epoch': 0.49, 'step': 900}, {'loss': 0.6651, 'learning_rate': 7.966101694915254e-05, 'epoch': 0.74, 'step': 1350}, {'eval_loss': 0.6362382769584656, 'eval_runtime': 20.882, 'eval_samples_per_second': 9.578, 'eval_steps_per_second': 0.623, 'epoch': 0.74, 'step': 1350}, {'loss': 0.6788, 'learning_rate': 3.389830508474576e-06, 'epoch': 0.99, 'step': 1800}, {'eval_loss': 0.633888304233551, 'eval_runtime': 20.8811, 'eval_samples_per_second': 9.578, 'eval_steps_per_second': 0.623, 'epoch': 0.99, 'step': 1800}, {'train_runtime': 9762.3204, 'train_

In [21]:
print(trainer.state.log_history)

[{'loss': 0.6567, 'learning_rate': 0.00023220338983050845, 'epoch': 0.25, 'step': 450}, {'eval_loss': 0.6495810151100159, 'eval_runtime': 20.8817, 'eval_samples_per_second': 9.578, 'eval_steps_per_second': 0.623, 'epoch': 0.25, 'step': 450}, {'loss': 0.6604, 'learning_rate': 0.0001559322033898305, 'epoch': 0.49, 'step': 900}, {'eval_loss': 0.6438479423522949, 'eval_runtime': 20.8671, 'eval_samples_per_second': 9.584, 'eval_steps_per_second': 0.623, 'epoch': 0.49, 'step': 900}, {'loss': 0.6651, 'learning_rate': 7.966101694915254e-05, 'epoch': 0.74, 'step': 1350}, {'eval_loss': 0.6362382769584656, 'eval_runtime': 20.882, 'eval_samples_per_second': 9.578, 'eval_steps_per_second': 0.623, 'epoch': 0.74, 'step': 1350}, {'loss': 0.6788, 'learning_rate': 3.389830508474576e-06, 'epoch': 0.99, 'step': 1800}, {'eval_loss': 0.633888304233551, 'eval_runtime': 20.8811, 'eval_samples_per_second': 9.578, 'eval_steps_per_second': 0.623, 'epoch': 0.99, 'step': 1800}, {'train_runtime': 9762.3204, 'train_

In [22]:
# # simple inference example
# q = 'Доставка пиццы Эшли стоит 15 долларов. Какова общая сумма, которую Эшли должна дать доставщику, если она хочет дать чаевые, равные 1/5 заказанной суммы?'
# lm_text=f'<SC6>Задача: {q}\nРешение:<extra_id_0>####<extra_id_1>'
# # answer: 18
# input_ids=torch.tensor([tokenizer.encode(lm_text)]).to(device)
# outputs=model.generate(input_ids,
#                        do_sample = True,
#                        top_p = 0.9,
#                        temperature=0.2,
#                        repetition_penalty = 2.0,
#                        min_length = 30,
#                        max_length = 450,
#                        eos_token_id=tokenizer.eos_token_id,
#                        early_stopping=True)
# tokenizer.decode(outputs[0][1:])

# Test

In [23]:
# peft_model_path="/kaggle/working/peft-checkpoint"

# peft_trainer.model.save_pretrained(peft_model_path)
# tokenizer.save_pretrained(peft_model_path)

In [24]:
# test no exec
# texts = [f'<SC6>Задача: {d["question"]}\nРешение:<extra_id_0>####<extra_id_1>' for d in train_data["test"]]
# answers = [d['answer'].split('####')[-1].strip() for d in train_data["test"]]
# N = 10
# correct_answers = 0
# for i, (text, answer) in enumerate(zip(texts[:N], answers[:N])):
#     input_ids=torch.tensor([tokenizer.encode(text)]).to(device)
#     outputs=model.generate(input_ids,
#                            eos_token_id=tokenizer.eos_token_id,
#                            early_stopping=True,
#                            top_p=0.9,
#                            temperature=0.2,
#                            repetition_penalty=2.0,
#                            min_length=50,
#                            max_length=512
#                           )
#     output = tokenizer.decode(outputs[0][1:])
#     print(f'=====[{i}]=====')
#     print('output:', output)
#     try:
#         output = output.split('<extra_id_1>')
#         if len(output) == 2:
#             pred = float(output[1].strip())
#         real = float(answer.split('####')[-1].strip())
#         error = abs(output - real)
#     except Exception as ex:
#         print(ex)
#         error = 1
#     if error < 1e-2:
#         correct_answers += 1
#         print('Successful')
#     else:
#         print('Wrong')

In [25]:
# correct_answers, train_data["test"].num_rows, correct_answers / train_data["test"].num_rows